# **Suspicious activities project**

## **Preprocess data**

### **Load data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import ast
import math
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go

In [ ]:
def get_dataframe(path):
    """ Read a dataframe from a path

    Input:
        -- path: string, path of the file

    Output:
        -- df: dataframe, containing the data

    """

    with open(path) as f:
        lines = f.readlines()
        tmp = pd.DataFrame(lines, columns=['input'])
        # Convert string to dict
        tmp['input'] = tmp['input'].apply(lambda x: ast.literal_eval(x))
        # tmp['input'] = tmp['input'].apply(lambda x: x.get('value'))
        df = pd.DataFrame(tmp['input'].to_list())
        df["date"] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
        return df 

clicks_path = "/content/drive/My Drive/streaming-project/offlineData/medium/clicks.txt"
displays_path = "/content/drive/My Drive/streaming-project/offlineData/medium/displays.txt"

df_clicks = get_dataframe(clicks_path)
df_displays = get_dataframe(displays_path)

In [ ]:
# Display clicks and displays dataframes
print('Clicks:')
display(df_clicks.head())
print('\nDisplays:')
display(df_displays.head())

Clicks:


,eventType,uid,timestamp,ip,impressionId,date
0,click,c82cea2b-932b-4853-9650-668e19d71739,1623489620,27.10.59.177,edf64522-7a1c-4091-8187-9a2db15bafe2,2021-06-12 09:20:20
1,click,75ac76f6-fe06-4058-8341-27d463abe98f,1623489621,177.152.232.109,b0799251-1e3f-4524-87fc-cc75329f3644,2021-06-12 09:20:21
2,click,08f5a4b9-ebab-4b75-aee6-ca9345c442ac,1623489622,236.250.33.183,5fa19bbf-51ca-4e4e-be33-eb704d96b02e,2021-06-12 09:20:22
3,click,71d678b5-85c5-452b-8a4a-bfbd93879f4a,1623489622,80.105.240.0,e1e1821e-5c96-4ae7-8e8b-5dffe8abe8ca,2021-06-12 09:20:22
4,click,3c70cea9-fb50-49fb-ad00-1ae4379f2ee010,1623489624,238.186.83.58,9126323b-1a11-4995-8d40-0a5d5c00fb69,2021-06-12 09:20:24



Displays:


,eventType,uid,timestamp,ip,impressionId,date
0,display,fb8b17bd-58d4-4c75-bc60-bfd70bb8ecc9,1623489620,145.87.127.251,86401821-e79d-43f5-b912-ebe88bd4f0f8,2021-06-12 09:20:20
1,display,a6a75171-e497-4eba-8e04-f85adb316baa,1623489620,77.157.20.22,bb90972e-6fe8-4301-b9d5-b1a5049c71c7,2021-06-12 09:20:20
2,display,5e7c4335-da38-421e-a025-9ce5dcb0196c,1623489620,34.176.201.86,db493100-e527-4ade-8894-30aa73d28bd5,2021-06-12 09:20:20
3,display,4f86e68b-ea8e-49e3-b945-e966c0e323cb,1623489620,10.187.11.82,2141082c-f1a8-481c-9d83-209326a348d4,2021-06-12 09:20:20
4,display,e48b767d-b9fe-4427-9698-40030b389cd3,1623489620,120.213.133.142,048520b5-710e-4aec-9f66-b859b6a0570a,2021-06-12 09:20:20


### **General informations of the datasets**

In [ ]:
def print_df_info(df):
    print(f'Nb of rows: {len(df)}')
    print(f"Nb unique uid: {len(df['uid'].unique())}")
    print(f"Nb unique ip: {len(df['ip'].unique())}")
    print(f"Nb unique impressionId: {len(df['impressionId'].unique())}")

**Information on clicks**

In [ ]:
print_df_info(df_clicks)

Nb of rows: 112213
Nb unique uid: 43923
Nb unique ip: 10000
Nb unique impressionId: 112213


**Information on displays**

In [ ]:
print_df_info(df_displays)

Nb of rows: 330254
Nb unique uid: 155983
Nb unique ip: 10001
Nb unique impressionId: 330254


For both datasets, there are as many impressionId as there are rows.

### **Filter on windows**

In [ ]:
def get_df_per_time(fdisplays, fclicks, month=False, height=540, width=1050):

    def filter_date(x, month=month):
        if month: return x.strftime("%Y-%m-%d %H:%M")
        else: return x.strftime("%H:%M")

    displays_per_time = fdisplays.copy()
    displays_per_time['time'] = displays_per_time['date'].apply(lambda x: filter_date(x))
    displays_per_time = displays_per_time.groupby(['time']).agg({'impressionId': 'count'})

    clicks_per_time = fclicks.copy()
    clicks_per_time ['time'] = clicks_per_time ['date'].apply(lambda x: filter_date(x))
    clicks_per_time = clicks_per_time.groupby(['time']).agg({'impressionId': 'count'})

    fig = go.Figure(
    data=[
    go.Bar(name='displays', x=displays_per_time.index, y=displays_per_time['impressionId']),
    go.Bar(name='clicks', x=clicks_per_time.index, y=clicks_per_time['impressionId']),
    ]
    )

    fig.update_layout(height=height, width=width, barmode='group', 
                    title={'text': 'Number clicks/displays at each minute', 'x': 0.5, 'y': 0.96},
                    margin=dict(b=60, t=60, l=65, r=20), 
                    # legend=dict(xanchor="right", x=1.11)
                    )

    fig.update_xaxes({'title': 'Datetime', 'title_standoff': 9})
    fig.update_yaxes({'title': 'Count', 'title_standoff': 10})
    fig.show()

**We want to choose a window that is representative of all our offline data**

In [ ]:
get_df_per_time(df_displays, df_clicks, month=True, width=1200, height=600)

In [ ]:
def filter_on_windows(df, start_time, window_size):
    ''' Filter the dataframe on a window time

    Inputs:
        -- df: dataframe, we want to filter
        -- start_time: datetime, starting time
        -- window_size: timedelta, difference time

    Output:
        -- filtered_df: dataframe, filtered on the window size and the start time

    '''

    filtered_df = df[df['date'].apply(lambda x: timedelta(minutes=0) <= x-start_time <= window_size)]
    return filtered_df

In [ ]:
# Define a start time and a window size
# Please choose a start_time that is in the dataset time slot
start_time = datetime(year=2021, month=6, day=13, hour=4, minute=40, second=0)
minutes = 5
window_size = timedelta(minutes=minutes)

# Filtered datasets 
# The last bar has less values because we take exactly 5 minutes
fclicks = filter_on_windows(df_clicks, start_time, window_size)
fdisplays = filter_on_windows(df_displays, start_time, window_size)
get_df_per_time(fdisplays, fclicks, month=False)

## **Suspicious activities and its outputs**

##**Patterns:**


> ### **Fraudulent UID:**
#### **1) Click Through Rate too high :** if for a given `uid`, its click through rate is higher than a certain percentage.
#### **2) UID that are associated with more than one IP :** if for a given `uid`, its number of unique `IP` is higher than 2 (included).
#### **3) Too many displays/clicks :** if for a given `uid`, there are too many displays or clicks.

> ### **Fraudulent IP**
#### **4) IP that are associated with too many `uid` :** if for a given IP, its number of unique `uid` is too high.



In [ ]:
# Store list of suspicious uids
suspicious_uids = []
# Store list of suspicious ips
suspicious_ips = []

##  **Fraudulent UID:**

### **Pattern 1: Click Through Rate too high** <br>
It corresponds to the number of clicks registered by your ad, divided by the number of times it was displayed and is expressed in percents (clicks ÷ impressions = CTR). For example, if you get 5 clicks and 100 impressions, your CTR is 5%. As hint, a normal CTR is around 10%

In [ ]:
# ouf of memory with merge
# impressions = imp_displays.merge(imp_clicks, how='left')

def get_nb_clicks(x, ref_clicks):
    ''' Count number of clicks for a given uid 

    Inputs:
        -- x: string, containing the uid
        -- ref_clicks: dataframe containing

    Output:
        -- number of clicks

    '''
    try : return ref_clicks.loc[x, 'nb_clicks']
    except : return 0


def get_displays_and_clicks(df_displays, df_clicks, _id, minutes=None):
    ''' Return a dataframe with number of displays, of clicks, CTR per _id where is whether uid or impressionId

    Inputs:
        -- df_displays: dataframe, of displays
        -- df_clicks: dataframe, of clicks
        -- _id: string, containing an id
        -- minutes: int (or None), determining the number of minutes

    Output:
        -- uid_displays: dataframe with nb_displays, nb_clicks, and the CTR per id (whether uid or impressionId)

    '''
    uid_displays = df_displays.groupby([_id]).agg({'eventType': 'count'}).reset_index()
    uid_displays = uid_displays.rename(columns={'eventType': 'nb_displays'})
    uid_clicks = df_clicks.groupby([_id]).agg({'eventType': 'count'})
    uid_clicks = uid_clicks.rename(columns={'eventType': 'nb_clicks'})

    # Compute CTR
    uid_displays['nb_clicks'] = uid_displays[_id].apply(lambda x: get_nb_clicks(x, ref_clicks=uid_clicks))
    uid_displays['CTR'] = uid_displays.apply(lambda x: x['nb_clicks']/x['nb_displays'], axis=1)
    if minutes: print(f"Mean ctr per {_id} over window {minutes} mins: {round(uid_displays['CTR'].mean()*100)}%")
    else: print(f"Mean ctr per {_id} over whole dataset : {round(uid_displays['CTR'].mean()*100)}%")
    return uid_displays

In [ ]:
# PER uid
print("#### CTR per UID ####")
uid_clicks_displays = get_displays_and_clicks(df_displays, df_clicks, "uid")
uid_fclicks_fdisplays = get_displays_and_clicks(fdisplays, fclicks, "uid", minutes=minutes)
print("\nSample of dataframe with CTR per uid on a fix window: \n", uid_fclicks_fdisplays.tail(3))

print("\n\n#### CTR as all clicks/displays ####")
print(f'Mean CTR over whole dataset: {round(len(df_clicks)/len(df_displays)*100,2)}')
print(f'Mean CTR over window {minutes} mins: {round(len(fclicks)/len(fdisplays)*100,2)}')

# # Per impressionId
# imp_displays = get_displays_and_clicks(df_displays, df_clicks, "impressionId")
# imp_fdisplays = get_displays_and_clicks(fdisplays, fclicks, "impressionId", minutes=minutes)

#### CTR per UID ####
Mean ctr per uid over whole dataset : 22%
Mean ctr per uid over window 5 mins: 14%

Sample of dataframe with CTR per uid on a fix window: 
                                        uid  nb_displays  nb_clicks  CTR
2271  ffc5c88c-bd7c-45a8-afd8-d2630f53f6f8            1          0  0.0
2272  ffd19ed3-cb36-4740-a65e-6737b8aac0d4            1          0  0.0
2273  ffdd04ee-1d0a-42a7-910d-07c29d772a22            1          0  0.0


#### CTR as all clicks/displays ####
Mean CTR over whole dataset: 33.98
Mean CTR over window 5 mins: 34.83


 We will put some filters to decide whether a uid is suspicious : 
 - `min_ctr` : having more than this threshold could be suspicious (this value can be changed)
 - `min_displays_ctr` : to ensure that we won't consider suspicious cases like 1click/1display or 2clicks/2displays


In [ ]:
# Histogram plot : we bucketize value from 0 to 1
bins = [i/10 for i in range(0, 12, 1)]
counts, bins = np.histogram(uid_fclicks_fdisplays.CTR, bins=bins)

fig = go.Figure(
    data=[
    go.Bar(name='ctr', x=bins, y=counts)
    ]
)

fig.update_layout(title={'text': 'Count uid per CTR', 'x': 0.5, 'y': 0.96}, 
                  height=450, 
                  width=700,
                  margin=dict(b=20, t=50, r=20, l=40)
                  )

fig.update_xaxes({'title': 'CTR', 'title_standoff': 12, 'tickvals': bins[:-1], 'range': [-0.1, 1.1]})
fig.update_yaxes({'title': 'Count', 'title_standoff': 8})
fig.show()

In [ ]:
# we set a threshold for a minimum number of displays
min_displays_ctr = math.ceil(3 * minutes)
min_ctr = 0.3
print(f'min_displays_ctr={min_displays_ctr} and min_ctr={min_ctr}')

suspicious_uid_CTR = uid_fclicks_fdisplays[(uid_fclicks_fdisplays['nb_displays']>=min_displays_ctr) & 
                                           (uid_fclicks_fdisplays['CTR']>min_ctr)
                                          ]['uid'].to_list()
print(f'There are {len(suspicious_uid_CTR)} uids that have a suspicious CTR')

# add these suspicious uids into the principal list without duplicates
suspicious_uids += [idx for idx in suspicious_uid_CTR if idx not in suspicious_uids]

min_displays_ctr=15 and min_ctr=0.3
There are 46 uids that have a suspicious CTR


### **Pattern 2: UID that are associated with more than one IP**

In [ ]:
def suspicious_uids_count_ip(df, min_ip_per_uid):
    ''' Return a dataframe, showing for a given uid, its associated list of ip and number of ip associated

    Input:
        -- df: dataframe of clicks or displays

    Output:
        -- tmp_df_uid: dataframe, updated

    '''
    tmp_df = df.groupby(['uid']).agg({'ip': 'unique'})
    tmp_df['nb_ip'] = tmp_df['ip'].apply(lambda x: len(x))
    tmp_df_uid = tmp_df[tmp_df['nb_ip']>min_ip_per_uid]
    return tmp_df_uid

In [ ]:
min_ip_per_uid = 1 
tmp_displays_uid = suspicious_uids_count_ip(fdisplays, min_ip_per_uid)
print(f"Sample of displays-dataframe where uids are associated with more than {min_ip_per_uid} IP: \n", tmp_displays_uid.head(3))
tmp_clicks_uid = suspicious_uids_count_ip(fclicks, min_ip_per_uid)
print(f"\nSample of clicks-dataframe where uids are associated with more than {min_ip_per_uid} IP: \n",tmp_clicks_uid.head(3))

suspicious_uids_ip = [uid for uid in tmp_displays_uid.index]
suspicious_uids_ip += [idx for idx in tmp_clicks_uid.index if idx not in suspicious_uids_ip]
print(f'\nThere are {len(suspicious_uids_ip)} uids that are associated with more than {min_ip_per_uid} IP')

# add these suspicious uids into the principal list without duplicates
suspicious_uids += [idx for idx in suspicious_uids_ip if idx not in suspicious_uids]

Sample of displays-dataframe where uids are associated with more than 1 IP: 
                                                                                      ip  nb_ip
uid                                                                                           
036953b2-3e9c-4149-8590-ddf9ccfdcb4d  [81.224.25.190, 93.33.89.253, 94.67.21.138, 14...     34
0596afbe-1808-4f81-8db8-9a14f93d394e                    [157.248.208.53, 227.128.94.77]      2
08aba05f-7247-4e79-a146-897c848fed53  [28.101.203.157, 79.147.5.3, 8.7.229.19, 210.1...     36

Sample of clicks-dataframe where uids are associated with more than 1 IP: 
                                                                                      ip  nb_ip
uid                                                                                           
036953b2-3e9c-4149-8590-ddf9ccfdcb4d  [127.75.152.233, 147.202.61.15, 119.28.169.99,...     24
08aba05f-7247-4e79-a146-897c848fed53  [126.191.208.58, 247.160.46.110, 201.128.238.2.

### **Pattern 3: UID where we have lots of displays**
By lots of displays we mean lots of impressionId here 

In [ ]:
# we set a threshold for a minimum number of displays/clicks
min_displays_per_uid = 4 * minutes
suspicious_uids_displays = uid_fclicks_fdisplays[uid_fclicks_fdisplays['nb_displays'] >= min_displays_per_uid]
print(f"Sample of dataframe where the uid counts more than {min_displays_per_uid} number of displays: \n\n {suspicious_uids_displays.head(3)}")
suspicious_uids_displays = suspicious_uids_displays['uid'].to_list()
print(f'\n\nThere are {len(suspicious_uids_displays)} suspicious UIDs where we have too many displays on a fix window.')

suspicious_uids += [uid for uid in suspicious_uids_displays if uid not in suspicious_uids]

Sample of dataframe where the uid counts more than 20 number of displays: 

                                       uid  nb_displays  nb_clicks       CTR
43   036953b2-3e9c-4149-8590-ddf9ccfdcb4d           34         24  0.705882
107  08aba05f-7247-4e79-a146-897c848fed53           36         22  0.611111
109  08f5a4b9-ebab-4b75-aee6-ca9345c442ac           51         19  0.372549


There are 50 suspicious UIDs where we have too many displays on a fix window.


##  **Fraudulent IP:**

### **Pattern 4: Number of unique visitor per IP too high ( --> IP that are associated with too many UID)**

In [ ]:
def suspicious_ips_count_uid(df, min_uid_per_ip):
    ''' Return a dataframe, showing for a given ip, its associated list of uids and number of uids associated

    Input:
        -- df: dataframe, of clicks or displays
        -- min_uids: int, the minimum number of uids

    Output:
        -- suspicious_ip: list, of suspicious ips

    '''
    tmp_df = df.groupby(['ip']).agg({'uid':'unique'})
    tmp_df['nb_uid'] = tmp_df['uid'].apply(lambda x: len(x))
    display('Nb unique uid per IP:', tmp_df['nb_uid'].value_counts())
    suspicious_ip = tmp_df[tmp_df['nb_uid'] >= min_uid_per_ip].index
    suspicious_ip = [idx for idx in suspicious_ip]
    print(f"\nNumber of suspicious IP that are associated with more than {min_uid_per_ip} uids : {len(suspicious_ip)}\n")
    return suspicious_ip

In [ ]:
# we set a threshold for a minimum number of uids
min_uid_per_ip = math.ceil(2*minutes)

print(f'For clicks :')
suspicious_ips_clicks = suspicious_ips_count_uid(fclicks, min_uid_per_ip)

print(f'\nFor displays :')
suspicious_ips_displays = suspicious_ips_count_uid(fdisplays, min_uid_per_ip)

suspicious_ips_uid = suspicious_ips_clicks
suspicious_ips_uid += [idx for idx in suspicious_ips_displays if idx not in suspicious_ips_uid]

print(f'\nThere are {len(suspicious_ips_uid)} unique IPs per UID on a fix window')

suspicious_ips += [ip for ip in suspicious_ips_uid if ip not in suspicious_ips]

For clicks :


'Nb unique uid per IP:'

1      1038
2        54
300       1
3         1
Name: nb_uid, dtype: int64


Number of suspicious IP that are associated with more than 10 uids : 1


For displays :


'Nb unique uid per IP:'

1      2622
2       513
3        54
4         9
5         2
300       1
Name: nb_uid, dtype: int64


Number of suspicious IP that are associated with more than 10 uids : 1


There are 1 unique IPs per UID on a fix window


## **Final results**

In [ ]:
print(f"Finally there are {len(suspicious_ips)} suspicious IPs and {len(suspicious_uids)} suspicious UIDs.")

Finally there are 1 suspicious IPs and 56 suspicious UIDs.


## **Suspicious `uid` and `ip` removal** <br>
To ensure that our pattern are corrects, we compute the CTR after removing all suspicious. <br>

We can see below that the global ctr (all clicks/displays) has decreased to almost 10%.

In [ ]:
def remove_suspicious(df, suspicious_ips, suspicious_uids):
    tmp = df[df['ip'].apply(lambda x: x not in suspicious_ips)]
    return tmp[tmp['uid'].apply(lambda x: x not in suspicious_uids)]

rf_displays = remove_suspicious(df_displays, suspicious_ips, suspicious_uids)
rf_fdisplays = remove_suspicious(fdisplays, suspicious_ips, suspicious_uids)
rf_clicks = remove_suspicious(df_clicks, suspicious_ips, suspicious_uids)
rf_fclicks = remove_suspicious(fclicks, suspicious_ips, suspicious_uids)

# PER uid
print("#### CTR per UID ####")
print(f'> Before removing suspicious:')
uid_displays = get_displays_and_clicks(df_displays, df_clicks, "uid")
uid_fdisplays = get_displays_and_clicks(fdisplays, fclicks, "uid", minutes=minutes)
print(f'\n> After removing suspicious: ')
uid_displays = get_displays_and_clicks(rf_displays, rf_clicks, "uid")
uid_fdisplays = get_displays_and_clicks(rf_fdisplays, rf_fclicks, "uid", minutes=minutes)

print("\n\n#### CTR as all clicks/displays ####\n")
print(f'> Before removing suspicious:')
print(f'If we take all clicks/displays {round(len(df_clicks)/len(df_displays)*100,2)}')
print(f'Over window {minutes} mins when we take all clicks/displays: {round(len(fclicks)/len(fdisplays)*100,2)}\n')
print(f'> After removing suspicious: ')
print(f'If we take all clicks/displays {round(len(rf_clicks)/len(rf_displays)*100,2)}')
print(f'Over window {minutes} mins when we take all clicks/displays: {round(len(rf_fclicks)/len(rf_fdisplays)*100,2)}\n')

#### CTR per UID ####
> Before removing suspicious:
Mean ctr per uid over whole dataset : 22%
Mean ctr per uid over window 5 mins: 14%

> After removing suspicious: 
Mean ctr per uid over whole dataset : 7%
Mean ctr per uid over window 5 mins: 0%


#### CTR as all clicks/displays ####

> Before removing suspicious:
If we take all clicks/displays 33.98
Over window 5 mins when we take all clicks/displays: 34.83

> After removing suspicious: 
If we take all clicks/displays 13.09
Over window 5 mins when we take all clicks/displays: 13.82



In [ ]:
labels = ['uid_CTR','uid_ip','uid_displays']
values = [len(suspicious_uid_CTR), len(suspicious_uids_ip), len(suspicious_uids_displays)]

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels, 
                             values=values, 
                             pull=[0, 0.2, 0]
                             )
                    ]
                )

fig.update_layout(title={'text': 'Suspicious uid repartition over pattern', 'x': 0.5, 'y': 0.96}, 
                  height=400, 
                  width=500,
                  margin=dict(b=0, t=50, r=10, l=10)
                  )
fig.show()

In [ ]:
labels = ['suspicious','unsuspicious']
all_uids = set(fclicks['uid'].unique()).union(set(fdisplays['uid'].unique()))

values = [len(suspicious_uids), len(all_uids)-len(suspicious_uids)]

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels, 
                             values=values, 
                             )
                    ]
                )

fig.update_layout(title={'text': 'Suspicious uid proportion', 'x': 0.5, 'y': 0.96}, 
                  height=400, 
                  width=500,
                  margin=dict(b=0, t=50, r=15, l=10)
                  )
fig.show()